In [141]:
from fyers_apiv3 import fyersModel
import pandas as pd
import datetime as dt
from fyers_apiv3.FyersWebsocket import data_ws
from datetime import datetime
import sqlite3
import time
import pickle
import concurrent.futures
from urllib.parse import urlparse,parse_qs
import sqlite3
import yfinance as yf
import pytz
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Semaphore
import datetime
from datetime import datetime, timedelta


In [142]:
from jupyterthemes import get_themes
from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('monokai')

In [143]:

def datestring_to_epoch(date_string):
    """
    Convert a date string in "YYYY-MM-DD" format to epoch time (Unix timestamp).

    Parameters:
    - date_string: A date string in "YYYY-MM-DD" format

    Returns:
    - Epoch time as an integer
    """
    dt = datetime.strptime(date_string, "%Y-%m-%d")
    epoch = datetime.utcfromtimestamp(0)
    delta = dt - epoch
    return int(delta.total_seconds())

# Example usage:
date_string = "2023-11-11"
epoch_time = datestring_to_epoch(date_string)
print(f"Date String: {date_string}")
print(f"Epoch Time: {epoch_time}")





def epoch_to_datetime(epoch_time):
    """
    Convert epoch time (Unix timestamp) to a datetime object in Indian timezone.

    Parameters:
    - epoch_time: Epoch time as an integer

    Returns:
    - datetime object in Indian timezone (without timezone info in string representation)
    """
    utc_datetime = datetime.utcfromtimestamp(epoch_time)
    utc_timezone = pytz.timezone('UTC')
    indian_timezone = pytz.timezone('Asia/Kolkata')

    utc_datetime = utc_timezone.localize(utc_datetime)
    indian_datetime = utc_datetime.astimezone(indian_timezone)

    return indian_datetime.replace(tzinfo=None)

# Example usage:
epoch_time = 1636641600  # Replace this with your epoch time
converted_datetime = epoch_to_datetime(epoch_time)
print(f"Epoch Time: {epoch_time}")
print(f"Converted Datetime (Indian Timezone): {converted_datetime}")

from datetime import datetime, timedelta

def slice_dates_into_chunks(start_date, end_date):
    """
    Slice two dates into 3-month chunks.

    Parameters:
    - start_date: Start date as a string in "YYYY-MM-DD" format
    - end_date: End date as a string in "YYYY-MM-DD" format

    Returns:
    - List of lists, where each sublist represents a 3-month chunk [start_date, end_date]
    """
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    chunks = []
    current_chunk_start = start_date

    while current_chunk_start < end_date:
        current_chunk_end = current_chunk_start + timedelta(days=90)
        
        # Check for the edge case where the chunk end date exceeds the end date
        if current_chunk_end > end_date:
            current_chunk_end = end_date

        chunks.append([current_chunk_start.strftime("%Y-%m-%d"), current_chunk_end.strftime("%Y-%m-%d")])

        current_chunk_start = current_chunk_end + timedelta(days=1)

    return chunks

# Example usage:
start_date = "2017-01-01"
end_date = "2023-12-11"
result = slice_dates_into_chunks(start_date, end_date)
print(len(result))




def BuildDataFrame(d1,d2,stock,resolution):
    Resultantdict={}
    FinalDf=pd.DataFrame()
    #Slice the dates into three month chunks
    date_list=slice_dates_into_chunks(d1,d2)
    date_list = [
    [datestring_to_epoch(date_string) for date_string in sublist]
    for sublist in date_list]
    #The fyers api part
    for i in range(len(date_list)):
        data = {
        "symbol":stock,
        "resolution":resolution,
        "date_format":"0",
        "range_from":date_list[i][0],
        "range_to":date_list[i][1],
        "cont_flag":"1"
        }
        response=fyers.history(data)
        if(response['code']==200 and response['s']=='ok'):
            current_df=pd.DataFrame(response['candles'])
            current_df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
            FinalDf = pd.concat([FinalDf, current_df])
            
    FinalDf['Datetime'] = FinalDf['Datetime'].apply(epoch_to_datetime)
    FinalDf.set_index('Datetime',inplace=True)
    return FinalDf        
        

Date String: 2023-11-11
Epoch Time: 1699660800
Epoch Time: 1636641600
Converted Datetime (Indian Timezone): 2021-11-11 20:10:00
28


In [144]:
client_id='CDAAGPIZ8E-100'
secret_id='MRF6IPN10E'
url='google.com'
response_type="code"
grant_type="authorization_code"

In [145]:
session=fyersModel.SessionModel(client_id=client_id,
                               secret_key=secret_id,
                               redirect_uri=url,
                               response_type=response_type,
                               grant_type=grant_type)
response=session.generate_authcode()
print(response)

https://api-t1.fyers.in/api/v3/generate-authcode?client_id=CDAAGPIZ8E-100&redirect_uri=google.com&response_type=code&state=None


In [6]:


url='https://api-t1.fyers.in/api/v3/google.com?s=ok&code=200&auth_code=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE2OTk4MTM1NzMsImV4cCI6MTY5OTg0MzU3MywibmJmIjoxNjk5ODEyOTczLCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYUzU1MDgzIiwib21zIjoiSzEiLCJoc21fa2V5IjoiNTNmYWQ4YjhiOGQzOTU3OGNkNDAxMmEzMzBmMjU2NTFlYTU2ZTIyYjA1MDM5OWEwZjllYzI2YTYiLCJub25jZSI6IiIsImFwcF9pZCI6IkNEQUFHUElaOEUiLCJ1dWlkIjoiMWMzNzJlZmEyMDNiNDk3MGFhZjdkMDA2ZmUyZWUyY2IiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.MbrEbsj-CBfTRLp4Ps2azpC_5KBBN2ZZ8S52j1ZDFzo&state=None'
parsed_url = urlparse(url)
query_params = parse_qs(parsed_url.query)

auth_code = query_params.get('auth_code', [None])[0]

print(auth_code)


eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkubG9naW4uZnllcnMuaW4iLCJpYXQiOjE2OTk4MTM1NzMsImV4cCI6MTY5OTg0MzU3MywibmJmIjoxNjk5ODEyOTczLCJhdWQiOiJbXCJ4OjBcIiwgXCJ4OjFcIiwgXCJ4OjJcIiwgXCJkOjFcIiwgXCJkOjJcIiwgXCJ4OjFcIiwgXCJ4OjBcIl0iLCJzdWIiOiJhdXRoX2NvZGUiLCJkaXNwbGF5X25hbWUiOiJYUzU1MDgzIiwib21zIjoiSzEiLCJoc21fa2V5IjoiNTNmYWQ4YjhiOGQzOTU3OGNkNDAxMmEzMzBmMjU2NTFlYTU2ZTIyYjA1MDM5OWEwZjllYzI2YTYiLCJub25jZSI6IiIsImFwcF9pZCI6IkNEQUFHUElaOEUiLCJ1dWlkIjoiMWMzNzJlZmEyMDNiNDk3MGFhZjdkMDA2ZmUyZWUyY2IiLCJpcEFkZHIiOiIwLjAuMC4wIiwic2NvcGUiOiIifQ.MbrEbsj-CBfTRLp4Ps2azpC_5KBBN2ZZ8S52j1ZDFzo


In [7]:
session.set_token(auth_code)
response=session.generate_token()
print(response)
df=pd.DataFrame()
df=response
tk=df['access_token']
fyers=fyersModel.FyersModel(client_id=client_id,is_async=False,token=tk,log_path="")

{'s': 'ok', 'code': 200, 'message': '', 'access_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTk4MTM1ODQsImV4cCI6MTY5OTgzNTQyNCwibmJmIjoxNjk5ODEzNTg0LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6ImFjY2Vzc190b2tlbiIsImF0X2hhc2giOiJnQUFBQUFCbFVSalFVN0c1SFlzb2RBa3AzVHZpOVJOOU9hNUVTZC0xYmRtMVZZeTJpN1lJN1lnUTMyYk1Vc0RlWVlFcVdWRnFIaFFsVXpUSEhkOFBDZ1RUZHJkNGZRSVEzdTJ6RUF5Qk45Tkhid2owalZfTlV1UT0iLCJkaXNwbGF5X25hbWUiOiJTT0hVTSBNVUxFWSIsIm9tcyI6IksxIiwiaHNtX2tleSI6IjUzZmFkOGI4YjhkMzk1NzhjZDQwMTJhMzMwZjI1NjUxZWE1NmUyMmIwNTAzOTlhMGY5ZWMyNmE2IiwiZnlfaWQiOiJYUzU1MDgzIiwiYXBwVHlwZSI6MTAwLCJwb2FfZmxhZyI6Ik4ifQ.7yY2HpZovRaBmk0_FoNJlNIIVzQBMlUQwqEv3KW6rsw', 'refresh_token': 'eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJhcGkuZnllcnMuaW4iLCJpYXQiOjE2OTk4MTM1ODQsImV4cCI6MTcwMTA0NTAyNCwibmJmIjoxNjk5ODEzNTg0LCJhdWQiOlsieDowIiwieDoxIiwieDoyIiwiZDoxIiwiZDoyIiwieDoxIiwieDowIl0sInN1YiI6InJlZnJlc2hfdG9rZW4iLCJhdF9oYXNoIjoiZ0FBQUFBQmxVUmpRVT

In [11]:
Stocks=pd.read_csv('ind_nifty200list.csv')
Stocks['Symbol']= str('NSE:')+Stocks['Symbol']+str('-EQ')
Securities=(Stocks['Symbol']).to_list()
Securities.append('NSE:NIFTY50-INDEX')
Securities.append('NSE:NIFTYBANK-INDEX')
Securities

['NSE:ABB-EQ',
 'NSE:ACC-EQ',
 'NSE:AUBANK-EQ',
 'NSE:ABBOTINDIA-EQ',
 'NSE:ADANIENSOL-EQ',
 'NSE:ADANIENT-EQ',
 'NSE:ADANIGREEN-EQ',
 'NSE:ADANIPORTS-EQ',
 'NSE:ADANIPOWER-EQ',
 'NSE:ATGL-EQ',
 'NSE:AWL-EQ',
 'NSE:ABCAPITAL-EQ',
 'NSE:ABFRL-EQ',
 'NSE:ALKEM-EQ',
 'NSE:AMBUJACEM-EQ',
 'NSE:APOLLOHOSP-EQ',
 'NSE:APOLLOTYRE-EQ',
 'NSE:ASHOKLEY-EQ',
 'NSE:ASIANPAINT-EQ',
 'NSE:ASTRAL-EQ',
 'NSE:AUROPHARMA-EQ',
 'NSE:DMART-EQ',
 'NSE:AXISBANK-EQ',
 'NSE:BAJAJ-AUTO-EQ',
 'NSE:BAJFINANCE-EQ',
 'NSE:BAJAJFINSV-EQ',
 'NSE:BAJAJHLDNG-EQ',
 'NSE:BALKRISIND-EQ',
 'NSE:BANDHANBNK-EQ',
 'NSE:BANKBARODA-EQ',
 'NSE:BANKINDIA-EQ',
 'NSE:BATAINDIA-EQ',
 'NSE:BERGEPAINT-EQ',
 'NSE:BEL-EQ',
 'NSE:BHARATFORG-EQ',
 'NSE:BHEL-EQ',
 'NSE:BPCL-EQ',
 'NSE:BHARTIARTL-EQ',
 'NSE:BIOCON-EQ',
 'NSE:BOSCHLTD-EQ',
 'NSE:BRITANNIA-EQ',
 'NSE:CGPOWER-EQ',
 'NSE:CANBK-EQ',
 'NSE:CHOLAFIN-EQ',
 'NSE:CIPLA-EQ',
 'NSE:COALINDIA-EQ',
 'NSE:COFORGE-EQ',
 'NSE:COLPAL-EQ',
 'NSE:CONCOR-EQ',
 'NSE:COROMANDEL-EQ',
 'NSE:CROMPTO

In [9]:
def BuildDataFrame(d1,d2,stock,resolution):
    Resultantdict={}
    FinalDf=pd.DataFrame()
    #Slice the dates into three month chunks
    date_list=slice_dates_into_chunks(d1,d2)
    date_list = [
    [datestring_to_epoch(date_string) for date_string in sublist]
    for sublist in date_list]
    #The fyers api part
    for i in range(len(date_list)):
        data = {
        "symbol":stock,
        "resolution":resolution,
        "date_format":"0",
        "range_from":date_list[i][0],
        "range_to":date_list[i][1],
        "cont_flag":"1"
        }
        response=fyers.history(data)
        if(response['code']==200 and response['s']=='ok'):
            current_df=pd.DataFrame(response['candles'])
            current_df.columns = ['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
            FinalDf = pd.concat([FinalDf, current_df])
            
    FinalDf['Datetime'] = FinalDf['Datetime'].apply(epoch_to_datetime)
    FinalDf.set_index('Datetime',inplace=True)
    return FinalDf        
        

In [10]:
fyers.history(data)

NameError: name 'data' is not defined

In [12]:
BuildDataFrame('2017-01-01','2017-12-01','NSE:SBIN-EQ',5)

,Open,High,Low,Close,Volume
Datetime,,,,,
2017-07-03 09:15:00,274.20,275.00,273.60,273.65,381913
2017-07-03 09:20:00,273.65,273.95,273.00,273.70,210877
2017-07-03 09:25:00,273.55,273.70,273.10,273.45,141147
2017-07-03 09:30:00,273.50,273.90,273.40,273.65,107297
2017-07-03 09:35:00,273.70,273.80,273.05,273.25,99746
...,...,...,...,...,...
2017-11-30 15:05:00,320.30,320.30,318.95,319.00,1678056
2017-11-30 15:10:00,319.10,320.25,318.50,320.25,1538644
2017-11-30 15:15:00,320.25,321.10,319.60,320.35,2158681


In [13]:
MasterDictionary = {}
d1 = '2017-01-01'  # Corrected month format
d2 = '2023-12-01'  # Corrected month format
times = [2, 1, 5, 15]

for t in times:
    MasterDictionary[t] = {}  
    for i in range(len(Securities)):
        key = Securities[i].replace('-EQ', '').replace('NSE:', '')
        MasterDictionary[t][key] = BuildDataFrame(d1, d2, Securities[i], t)
        print(f"Appended {Securities[i]} for {t} minute timeframe")

# Clear output message
print("Data collection and appending to MasterDictionary completed.")


Appended NSE:ABB-EQ for 2 minute timeframe
Appended NSE:ACC-EQ for 2 minute timeframe
Appended NSE:AUBANK-EQ for 2 minute timeframe
Appended NSE:ABBOTINDIA-EQ for 2 minute timeframe
Appended NSE:ADANIENSOL-EQ for 2 minute timeframe
Appended NSE:ADANIENT-EQ for 2 minute timeframe
Appended NSE:ADANIGREEN-EQ for 2 minute timeframe
Appended NSE:ADANIPORTS-EQ for 2 minute timeframe
Appended NSE:ADANIPOWER-EQ for 2 minute timeframe
Appended NSE:ATGL-EQ for 2 minute timeframe
Appended NSE:AWL-EQ for 2 minute timeframe
Appended NSE:ABCAPITAL-EQ for 2 minute timeframe
Appended NSE:ABFRL-EQ for 2 minute timeframe
Appended NSE:ALKEM-EQ for 2 minute timeframe
Appended NSE:AMBUJACEM-EQ for 2 minute timeframe
Appended NSE:APOLLOHOSP-EQ for 2 minute timeframe
Appended NSE:APOLLOTYRE-EQ for 2 minute timeframe
Appended NSE:ASHOKLEY-EQ for 2 minute timeframe
Appended NSE:ASIANPAINT-EQ for 2 minute timeframe
Appended NSE:ASTRAL-EQ for 2 minute timeframe
Appended NSE:AUROPHARMA-EQ for 2 minute timeframe
A

KeyError: 'Datetime'

In [34]:
MasterDictionary.keys()

dict_keys([2, 1])

In [24]:
with open('MasterDictionary.pkl', 'wb') as file:
    pickle.dump(MasterDictionary, file)

print("MasterDictionary saved as MasterDictionary.pkl")

MasterDictionary saved as MasterDictionary.pkl


In [32]:
import pickle

# Load MasterDictionary from the pickle file
with open('MasterDictionary.pkl', 'rb') as file:
    loaded_MasterDictionary = pickle.load(file)

In [33]:
loaded_MasterDictionary[1]['ABB']

,Open,High,Low,Close,Volume
Datetime,,,,,
2017-07-03 09:15:00,1452.0,1453.7,1452.00,1453.7,11
2017-07-03 09:16:00,1453.7,1453.7,1450.60,1450.6,148
2017-07-03 09:17:00,1450.6,1452.8,1450.60,1452.8,70
2017-07-03 09:18:00,1452.8,1452.8,1452.80,1452.8,19
2017-07-03 09:19:00,1452.8,1453.2,1452.00,1452.0,38
...,...,...,...,...,...
2023-11-12 19:12:00,4230.0,4234.9,4229.00,4229.0,146
2023-11-12 19:13:00,4229.0,4233.0,4228.75,4232.0,482
2023-11-12 19:14:00,4232.0,4235.0,4227.20,4234.9,596


In [ ]:
#MasterDictionary = {}
d1 = '2017-01-01'  # Corrected month format
d2 = '2023-12-01'  # Corrected month format
times = [1, 5, 15]

for t in times:
    MasterDictionary[t] = {}  
    for i in range(len(Securities)):
        key = Securities[i].replace('-EQ', '').replace('NSE:', '')
        
        # Check if the key already exists in MasterDictionary[t]
        if key not in MasterDictionary[t]:
            # If the key does not exist, run the function and update the dictionary
            MasterDictionary[t][key] = BuildDataFrame(d1, d2, Securities[i], t)
            print(f"Appended {Securities[i]} for {t} minute timeframe")
        else:
            # If the key already exists, print a message indicating that it's skipped
            print(f"Skipped {Securities[i]} for {t} minute timeframe as it already exists")

# Clear output message
print("Data collection and appending to MasterDictionary completed.")


In [35]:

conn = sqlite3.connect('HistoricData.db')

In [46]:
%%time
for timeframe, stock_data in MasterDictionary.items():
    for stockname, df in stock_data.items():
        table_name = f"{stockname}_{timeframe}"
        df.to_sql(table_name, conn, index=True, if_exists='replace')

CPU times: total: 11min 17s
Wall time: 14min 28s


In [56]:
def get_dataframe(stockname, timeframe, conn):
    # Use parameterized query to handle special characters in table name
    table_name = f'"{stockname}_{timeframe}"'
    query = f"SELECT Datetime, Close FROM {table_name}"
    
    # Specify the database connection in the function parameters
    df = pd.read_sql_query(query, conn)
    
    # Convert 'Datetime' column to datetime type
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    
    # Set 'Datetime' as the index
    #df.set_index('Datetime', inplace=True)
    
    # Rename 'Close' column to the stock name
    df.rename(columns={'Close': stockname}, inplace=True)
    
    return df


In [116]:
outer_merged=dfs[0]

# Iterate over the remaining DataFrames and merge them one by one
for i, df in enumerate(dfs[1:],start=1):
    # Merge based on a common column with 'outer' join
    outer_merged = pd.merge(outer_merged, df, on=index_col, how='left')
    outer_merged.drop_duplicates('Datetime',inplace=True)
    # Print the iteration number
    print(f'Iteration {i} done')

Iteration 1 done
Iteration 2 done
Iteration 3 done
Iteration 4 done
Iteration 5 done
Iteration 6 done
Iteration 7 done
Iteration 8 done
Iteration 9 done
Iteration 10 done
Iteration 11 done
Iteration 12 done
Iteration 13 done
Iteration 14 done
Iteration 15 done
Iteration 16 done
Iteration 17 done
Iteration 18 done
Iteration 19 done
Iteration 20 done
Iteration 21 done
Iteration 22 done
Iteration 23 done
Iteration 24 done
Iteration 25 done
Iteration 26 done
Iteration 27 done
Iteration 28 done
Iteration 29 done
Iteration 30 done
Iteration 31 done
Iteration 32 done
Iteration 33 done
Iteration 34 done
Iteration 35 done
Iteration 36 done
Iteration 37 done
Iteration 38 done
Iteration 39 done
Iteration 40 done
Iteration 41 done
Iteration 42 done
Iteration 43 done
Iteration 44 done
Iteration 45 done
Iteration 46 done
Iteration 47 done
Iteration 48 done
Iteration 49 done
Iteration 50 done
Iteration 51 done
Iteration 52 done
Iteration 53 done
Iteration 54 done
Iteration 55 done
Iteration 56 done
I

In [119]:
outer_merged['Datetime'] = pd.to_datetime(outer_merged['Datetime'])
outer_merged.set_index('Datetime',inplace=True)

In [120]:
backupdf=outer_merged

In [121]:
import datetime

# Convert index to datetime

# Sort by datetime index
outer_merged.sort_index(inplace=True)

# Filter rows where the time is less than or equal to 3:30 PM
valid_datetime_mask = outer_merged.index.time <= datetime.time(15, 30)
outer_merged = outer_merged[valid_datetime_mask]


In [123]:
outer_merged.reset_index(inplace=True)
outer_merged.rename(columns={'Datetime': 'date'}, inplace=True)
outer_merged

C:\Users\Sohum\AppData\Local\Temp\ipykernel_13716\2751189960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  outer_merged.rename(columns={'Datetime': 'date'}, inplace=True)


,date,TATACONSUM,DLF,PGHH,ASIANPAINT,JINDALSTEL,MARUTI,GAIL,TATAMOTORS,AMBUJACEM,...,INDUSTOWER,SBILIFE,LT,PIIND,INFY,PEL,DMART,NAUKRI,MUTHOOTFIN,JUBLFOOD
0,2017-07-03 09:15:00,151.60,191.50,8050.00,1109.60,123.30,7217.00,90.54,427.45,246.60,...,383.15,NaN,1123.37,836.45,469.48,2723.10,812.65,1048.9,457.70,94.72
1,2017-07-03 09:17:00,151.25,190.80,8005.20,1110.55,123.05,7220.00,90.00,427.40,246.00,...,378.85,NaN,1119.27,836.75,468.80,2724.16,808.70,1038.0,457.35,94.61
2,2017-07-03 09:19:00,151.35,190.65,8005.20,1110.00,123.05,7254.00,90.17,427.35,246.00,...,381.75,NaN,1122.60,837.95,469.05,2718.10,809.90,1033.0,458.60,94.73
3,2017-07-03 09:21:00,151.50,190.85,8005.20,1107.65,123.60,7252.45,90.11,426.75,246.40,...,380.70,NaN,1121.30,836.00,468.90,2710.87,813.95,1038.0,456.05,94.80
4,2017-07-03 09:23:00,151.45,190.65,8030.75,1107.70,123.20,7276.00,90.21,426.85,247.15,...,380.55,NaN,1120.67,834.65,469.20,2702.76,813.00,1041.7,456.50,94.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295026,2023-11-10 15:21:00,912.10,604.75,17838.90,3078.90,636.95,10395.00,125.70,651.50,417.90,...,183.25,1355.45,3029.90,3681.95,1369.35,946.85,3796.95,4509.9,1273.90,507.95
295027,2023-11-10 15:23:00,913.75,604.95,17789.20,3079.95,637.80,10396.20,125.70,651.90,417.70,...,183.30,1355.50,3034.10,3687.00,1370.00,947.05,3797.90,4511.0,1273.00,507.75
295028,2023-11-10 15:25:00,913.45,604.80,17842.90,3080.20,637.15,10400.00,125.75,651.90,418.00,...,183.20,1354.65,3034.75,3681.95,1370.65,947.05,3798.00,4511.9,1271.80,508.10
295029,2023-11-10 15:27:00,914.00,605.90,17721.10,3081.90,636.40,10396.10,125.50,651.80,418.30,...,183.45,1354.90,3033.20,3688.05,1370.15,945.65,3799.00,4515.9,1269.05,508.45


In [4]:
PreDataFrame=pd.read_csv('nifty_100_updated.csv')
PreDataFrame

,date,TATACONSUM,DLF,PGHH,ASIANPAINT,JINDALSTEL,MARUTI,GAIL,TATAMOTORS,AMBUJACEM,...,INDUSTOWER,SBILIFE,LT,PIIND,INFY,PEL,DMART,NAUKRI,MUTHOOTFIN,JUBLFOOD
0,2021-10-04 09:15:00+05:30,803.60,418.65,14162.15,3192.00,408.00,7141.20,109.25,337.30,404.55,...,312.00,1212.75,1689.05,3159.60,1671.05,2671.95,4423.90,6481.50,1534.60,819.10
1,2021-10-04 09:17:00+05:30,805.15,417.65,14155.10,3190.20,407.70,7142.00,109.15,337.05,403.75,...,312.65,1211.50,1688.25,3161.25,1676.35,2676.15,4386.70,6466.20,1539.20,819.95
2,2021-10-04 09:19:00+05:30,806.90,420.00,14101.10,3192.05,406.55,7147.65,108.75,338.60,403.60,...,311.80,1212.60,1694.30,3164.45,1673.80,2671.45,4376.75,6461.55,1530.15,822.60
3,2021-10-04 09:21:00+05:30,806.90,419.30,14128.80,3186.75,406.30,7169.00,108.60,339.30,403.85,...,311.80,1210.65,1698.30,3162.50,1675.25,2672.50,4370.45,6467.60,1531.50,823.75
4,2021-10-04 09:23:00+05:30,805.05,418.35,14084.00,3182.60,405.45,7201.00,108.95,341.00,403.25,...,311.00,1213.35,1700.00,3160.00,1674.65,2675.95,4333.30,6459.70,1532.60,827.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92679,2023-09-28 15:21:00+05:30,879.45,520.65,17594.75,3175.35,683.45,10573.45,121.95,614.80,420.00,...,189.80,1290.30,3012.85,3392.30,1439.85,1019.00,3674.80,4175.95,1212.05,540.35
92680,2023-09-28 15:23:00+05:30,880.50,520.35,17556.20,3176.15,684.20,10611.50,121.70,615.00,420.10,...,190.50,1291.00,3013.20,3395.30,1441.15,1018.60,3674.25,4174.50,1213.10,540.10
92681,2023-09-28 15:25:00+05:30,880.05,520.00,17565.40,3177.65,684.60,10593.10,121.95,615.20,420.60,...,191.15,1290.45,3016.95,3394.90,1441.35,1019.55,3674.00,4179.95,1213.00,542.45
92682,2023-09-28 15:27:00+05:30,878.10,521.00,17586.00,3179.00,685.20,10595.00,122.00,615.00,421.30,...,191.35,1290.00,3025.00,3408.50,1443.00,1019.45,3658.90,4178.95,1210.95,540.95


In [136]:
import pandas as pd

def compare_dataframes(df1, df2):
    # Compare shapes
    if df1.shape != df2.shape:
        print("Shapes are different:")
        print(f"DataFrame 1 shape: {df1.shape}")
        print(f"DataFrame 2 shape: {df2.shape}")
    else:
        print("Shapes are the same.")

    # Compare column names
    if list(df1.columns) != list(df2.columns):
        print("\nColumn names are different:")
        print(f"DataFrame 1 columns: {list(df1.columns)}")
        print(f"DataFrame 2 columns: {list(df2.columns)}")
    else:
        print("\nColumn names are the same.")

    # Compare data types
    if not df1.dtypes.equals(df2.dtypes):
        print("\nData types are different:")
        print(f"DataFrame 1 data types: {df1.dtypes}")
        print(f"DataFrame 2 data types: {df2.dtypes}")
    else:
        print("\nData types are the same.")

    # Compare values
    if not df1.equals(df2):
        print("\nValues are different.")
    else:
        print("\nValues are the same.")

    # Additional comparisons based on specific requirements can be added here

# Example usage:
# Assuming df1 and df2 are your DataFrames
# For example:
# df1 = ...
# df2 = ...
compare_dataframes(PreDataFrame, outer_merged)


Shapes are different:
DataFrame 1 shape: (92684, 97)
DataFrame 2 shape: (295031, 97)

Column names are the same.

Data types are the same.

Values are different.


In [139]:
outer_merged.to_csv('FinalFinal.csv')

In [140]:
outer_merged

,date,TATACONSUM,DLF,PGHH,ASIANPAINT,JINDALSTEL,MARUTI,GAIL,TATAMOTORS,AMBUJACEM,...,INDUSTOWER,SBILIFE,LT,PIIND,INFY,PEL,DMART,NAUKRI,MUTHOOTFIN,JUBLFOOD
0,2017-07-03 09:15:00,151.60,191.50,8050.00,1109.60,123.30,7217.00,90.54,427.45,246.60,...,383.15,NaN,1123.37,836.45,469.48,2723.10,812.65,1048.9,457.70,94.72
1,2017-07-03 09:17:00,151.25,190.80,8005.20,1110.55,123.05,7220.00,90.00,427.40,246.00,...,378.85,NaN,1119.27,836.75,468.80,2724.16,808.70,1038.0,457.35,94.61
2,2017-07-03 09:19:00,151.35,190.65,8005.20,1110.00,123.05,7254.00,90.17,427.35,246.00,...,381.75,NaN,1122.60,837.95,469.05,2718.10,809.90,1033.0,458.60,94.73
3,2017-07-03 09:21:00,151.50,190.85,8005.20,1107.65,123.60,7252.45,90.11,426.75,246.40,...,380.70,NaN,1121.30,836.00,468.90,2710.87,813.95,1038.0,456.05,94.80
4,2017-07-03 09:23:00,151.45,190.65,8030.75,1107.70,123.20,7276.00,90.21,426.85,247.15,...,380.55,NaN,1120.67,834.65,469.20,2702.76,813.00,1041.7,456.50,94.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295026,2023-11-10 15:21:00,912.10,604.75,17838.90,3078.90,636.95,10395.00,125.70,651.50,417.90,...,183.25,1355.45,3029.90,3681.95,1369.35,946.85,3796.95,4509.9,1273.90,507.95
295027,2023-11-10 15:23:00,913.75,604.95,17789.20,3079.95,637.80,10396.20,125.70,651.90,417.70,...,183.30,1355.50,3034.10,3687.00,1370.00,947.05,3797.90,4511.0,1273.00,507.75
295028,2023-11-10 15:25:00,913.45,604.80,17842.90,3080.20,637.15,10400.00,125.75,651.90,418.00,...,183.20,1354.65,3034.75,3681.95,1370.65,947.05,3798.00,4511.9,1271.80,508.10
295029,2023-11-10 15:27:00,914.00,605.90,17721.10,3081.90,636.40,10396.10,125.50,651.80,418.30,...,183.45,1354.90,3033.20,3688.05,1370.15,945.65,3799.00,4515.9,1269.05,508.45
